# Mel-Spectrogram

Quelques écoutes pour avoir une idée de la taille des spectrogrammes en E/S. Reconstruction audio par G&L

In [2]:
import numpy as np
from scipy.io import wavfile
import scipy
import pandas as pd
import librosa
import librosa.display
import time

import matplotlib.pyplot as plt
%matplotlib widget
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import Audio

from synth import dexed

specs_cmap = 'magma'  # Spectrograms default colormap

In [2]:
dexed_preset_db = dexed.PresetDatabase()
print("{} presets available".format(dexed_preset_db.get_nb_presets()))
chosen_algo = 5
preset_indexes = dexed_preset_db.get_preset_indexes_for_algorithm(chosen_algo)
print("{} presets using algorithm {}".format(len(preset_indexes), chosen_algo))

30293 presets available
3671 presets using algorithm 5


### Choix des paramètres du spectrogramme
* 22,050kHz par défaut dans librosa, d'autres ont choisi ça (on peut rester là-dessus...)
* Défault librosa
    * fft largeur 2048 (9,3ms) hopsize 512
* FlowSynth 2019
    * fft largeur 2048, hop 1024, 128 mel-bins de 30 à 11kHz
        * 128 mel-bins ça permet difficilement de reconstruire un son de DX7... (déjà sans AE)
    * log, normalized
    * corpus-wide zero-mean, unit-variance normalization on each spectrogram (based on the train dataset)
* Roche 2020
    * Dataset NSynth : fft largeur 1024, hop 512, 513 lin freq bins de la STFT basique
    * Dataset Arturia : fft 2048 (hop toujours 50% ?)
    * log, energy-normalized, threshold -100dB
* Wavenet (modèle baseline avec spectrogramme) 2017
    * fft largeur 1024, fft hop 256
    * STFT linéaire ??? (à confirmer)
    * log magnitude du power spectrum
    * Constant-Q transform pour *l'affichage* des résultats
    * *Ou alors, encoder wavenet sur waveform directement*
    
#### Problème de la taille du spectrogramme

### Utiliser un spectrogramme plutôt qu'un mel-spectrogramme ?
* $+$ représentation beaucoup plus fine du son, surtout pour un synthé comme le DX7
* $-$ pas d'échelle log pour les harmoniques, peut être un problème pour un CNN
* $+/-$ image à traiter est plus grande

In [9]:
t0 = time.time()
# Test pour voir si rendu temps-réel est viable. 5ms ouverture + 13ms pour 5s d'audio
for i in range(20):  # + rapide si petite boucle... (14ms total)
    dexed_renderer = dexed.Dexed()

    #idx = preset_indexes[1363]
    idx = 2247
    preset = dexed_preset_db.get_preset_values(idx, plugin_format=True)
    dexed_renderer.assign_preset(preset)
    dexed_renderer.set_all_oscillators_on()
    dexed_renderer.set_default_general_filter_and_tune_params()
    dexed_renderer.prevent_SH_LFO()
    x_wav = dexed_renderer.render_note(60, 100)
print("Temps moyen pour ouvrir dexed (nouvelle classe) et générer l'audio = {:.1f}ms\n".format(1000.0*(time.time() - t0)/20.0))

Fs = dexed_renderer.Fs

print("Input audio")
Audio(x_wav, rate=Fs)

IndexError: index 78256 is out of bounds for axis 0 with size 30293

In [4]:
# PyTorch STFT test - for comparison
import torch
import torch.fft
import importlib
import utils.audio
importlib.reload(utils.audio)

# Default values for librosa (mel)spectrogram: 2048
fft_width = 1024
print("fft_width : {:.1f}ms ({} linear freq bins per time step).".format(1000.0*fft_width/Fs, fft_width//2+1))
print("Freq resolution = {:.1f}Hz".format(Fs / fft_width))
fft_hop = 256
x_dB_min_th = -130.0  # Valeur min du log-spectre

dynamic_range_dB = 100  # Deprecated

get_spec = utils.audio.Spectrogram(fft_width, fft_hop, x_dB_min_th)
get_mel_spec = utils.audio.MelSpectrogram(fft_width, fft_hop, x_dB_min_th, 257, 22050)

print("Spectrogram norm factor (Hann window) = {}".format(get_spec.spectrogram_norm_factor))

x_test = np.cos(2*np.pi*0.005 * np.arange(0, x_wav.shape[0]))  # Use this to check spectrogram norm
x_test = x_wav

spectrogram = get_spec(x_test)
mel_spectrogram = get_mel_spec(x_test)
inverse_mel_spectrogram = get_mel_spec.mel_dB_to_STFT(mel_spectrogram)
inverse_mel_spectrogram = get_mel_spec.linear_to_log_scale(torch.Tensor(inverse_mel_spectrogram))

print("PyTorch Spectrogram size = {}".format(spectrogram.size()))

fig, axes = plt.subplots(2, 2, figsize=(8,8))
axes[0][0].plot(x_wav)
axes[0][1].set(title="PyTorch STFT (dB)\nMin={:.1f} max={:.1f}".format(spectrogram.numpy().min(), spectrogram.numpy().max()),
       xlabel='Time [s]', ylabel='Freq [Hz]')
librosa.display.specshow(spectrogram.numpy(), shading='flat', ax=axes[0][1], cmap=specs_cmap)
axes[1][0].set(title="PyTorch STFT + Librosa Mel (dB)\nMin={:.1f} max={:.1f}"
            .format(mel_spectrogram.numpy().min(), mel_spectrogram.numpy().max()))
librosa.display.specshow(mel_spectrogram.numpy(), shading='flat', ax=axes[1][0], cmap=specs_cmap)
axes[1][1].set(title="STFT from inverse Mel (dB)\nMin={:.1f} max={:.1f}"
            .format(inverse_mel_spectrogram.numpy().min(), inverse_mel_spectrogram.numpy().max()))
librosa.display.specshow(inverse_mel_spectrogram.numpy(), shading='flat', ax=axes[1][1], cmap=specs_cmap)
fig.tight_layout()

fft_width : 46.4ms (513 linear freq bins per time step).
Freq resolution = 21.5Hz
Spectrogram norm factor (Hann window) = 511.5
PyTorch Spectrogram size = torch.Size([513, 433])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# SciPy/Librosa spectrograms
fft_width = 2048  # easy config for mel-inverse
# STFT array: last axis correspond to segment times
print("Original signal length = {}".format(x_wav.shape[0]))
f_stft, t_stft, x_stft = scipy.signal.stft(x_wav, fs=Fs, nperseg=fft_width, noverlap=fft_width-fft_hop, padded=True)
print("spectrogram fft hop  : {:.1f}ms. {} time steps for {:.1f}s audio (padded)".format(1000.0*fft_hop/Fs, len(t_stft), x_wav.shape[0]/Fs))
print("STFT size : {}".format(x_stft.shape))
log_x_stft = np.maximum(20.0 * np.log10(np.abs(x_stft)), x_dB_min_th)

# Mel-spectrogram (and filter bank alone, for visualization)
n_mels = 256
f_mel_min = 30  # flow synth
f_mel_max = 11e3
melfb = librosa.filters.mel(Fs, fft_width, n_mels=n_mels, fmin=f_mel_min, fmax=f_mel_max)
mel_spectrogram = librosa.feature.melspectrogram(x_wav, sr=Fs, n_fft=fft_width, hop_length=fft_hop,
                                                 n_mels=n_mels, fmin=f_mel_min, fmax=f_mel_max)
log_mel_spectrogram = np.maximum(20.0 * np.log10(np.abs(mel_spectrogram)), x_dB_min_th)

# Reconstruction à partir du mel-spectro - prend un temps énorme. Ne converge pas toujours !
x_stft_from_mel = librosa.feature.inverse.mel_to_stft(mel_spectrogram, sr=Fs, n_fft=fft_width,
                                                      fmin=f_mel_min, fmax=f_mel_max)
log_x_stft_from_mel = np.maximum(20.0 * np.log10(np.abs(x_stft_from_mel)), x_dB_min_th)
print("Reconstructed STFT size : {}".format(x_stft_from_mel.shape))
# Reconstruction Audio via G&L - init avec la vraie phase à retrouver
x_stft_from_mel_with_phase = x_stft_from_mel * np.exp(1j * np.angle(x_stft))
print("Reconstructed STFT w/phase size : {}".format(x_stft_from_mel_with_phase.shape))
x_wav_from_mel = librosa.griffinlim(x_stft_from_mel_with_phase)
print("Mel-reconstructed signal length = {}".format(x_wav_from_mel.shape[0]))

fig, axes = plt.subplots(3, 2, figsize=(9, 8))
axes[0][0].set(title="Input Audio", xlabel="Time [s]")
axes[0][0].plot(np.arange(0, x_wav.shape[0]) / Fs, x_wav)
axes[1][0].set(title="STFT Spectrogram (log module)", xlabel='Time [s]', ylabel='Freq [Hz]')
im = axes[1][0].pcolormesh(t_stft, f_stft, log_x_stft, shading='gouraud')
fig.colorbar(im, ax=axes[1][0], orientation='vertical', format='%.0f dB')
im = librosa.display.specshow(melfb, x_axis='linear', ax=axes[2][0], cmap='nipy_spectral')
axes[2][0].set(ylabel='output index', title='Mel filter bank')
fig.colorbar(im, ax=axes[2][0], orientation='vertical')
im = librosa.display.specshow(log_mel_spectrogram, x_axis='time', y_axis='mel', sr=Fs, ax=axes[2][1], fmin=f_mel_min, fmax=f_mel_max)
fig.colorbar(im, ax=axes[2][1], orientation='vertical', format='%.0f dB')
axes[2][1].set(title="Mel Spectrogram - {} bins".format(n_mels))
axes[1][1].set(title="Reconstructed STFT Spectrogram (??)", xlabel='Time [s]', ylabel='Freq [Hz]')
im = axes[1][1].pcolormesh(t_stft, f_stft, log_x_stft_from_mel, shading='gouraud')
fig.colorbar(im, ax=axes[1][1], orientation='vertical', format='%.0f dB')
axes[0][1].set(title="Reconstructed Audio", xlabel="Time [s]")
axes[0][1].plot(np.arange(0, x_wav.shape[0]) / Fs, x_wav_from_mel)
fig.tight_layout()

Original signal length = 110592
spectrogram fft hop  : 11.6ms. 433 time steps for 5.0s audio (padded)
STFT size : (1025, 433)


<ipython-input-5-f91c1861193c>:22: RuntimeWarning: divide by zero encountered in log10
  log_x_stft_from_mel = np.maximum(20.0 * np.log10(np.abs(x_stft_from_mel)), x_dB_min_th)


Reconstructed STFT size : (1025, 433)
Reconstructed STFT w/phase size : (1025, 433)
Mel-reconstructed signal length = 221184


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gwendal/anaconda3/lib/python3.8/site-packages/librosa/display.py:974: MatplotlibDeprecationWarning: The 'basey' parameter of __init__() has been renamed 'base' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)
/home/gwendal/anaconda3/lib/python3.8/site-packages/librosa/display.py:974: MatplotlibDeprecationWarning: The 'linthreshy' parameter of __init__() has been renamed 'linthresh' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)


ValueError: x and y must have same first dimension, but have shapes (110592,) and (221184,)

In [ ]:
print("Reconstructed audio")
Audio(x_wav_from_mel, rate=Fs)

## Dataset presets, audio and spectrograms

Ceux qui risquent de poser problème :
* Sons harmoniques mais qui partent en sucette :
    * 3104
* Sons harmoniques qui partent en sucette au release :
    * 5833
* SFX, sons intéressants mais pas harmoniques (pourraient aider le NN à bien comprendre comment fonctionne Dexed ?)
    * 2520, 2522
    * 3103, 3105, 3107, 3109, 3110, 3112, 3115, 3116, 3118, ...
    * 22468
    * 75989
    * 76105
    * 245768
* Kicks / percus (idem, peut-être intéressants)
    * 81538
    * 246879
* Presque zéro :
    * 54035
    * 2555 (sorte de petit kick puis bruit...)
* Gros n'imp, limite inécoutable (par UID) :
    * 3111
    * 71429
    * 74146
    * 75340
    * 78256
    * 81660, 81662
    * 245768


In [74]:
from data import dataset
import matplotlib.pyplot as plt
import importlib
import config
importlib.reload(dataset)

dexed_dataset = dataset.DexedDataset(note_duration=config.model.note_duration, n_fft=config.model.stft_args[0],
                                     fft_hop=config.model.stft_args[1], n_mel_bins=config.model.mel_bins)
print(dexed_dataset)

# Test sample can be loaded here by UID
x_wav, Fs = dexed_dataset._get_wav_file(7814)
Audio(x_wav, rate=Fs)

Dataset of 30293/30293 Dexed presets. 144 learnable synth params, 11 fixed params. Mel Spectrogram items, size=torch.Size([1, 257, 347]), min=-120.0dB, normalization:min_max


In [30]:
# TODO function propre de fig - en package utils.figu
# Partial display of a dataloader batch
from torch.utils.data import DataLoader

import utils.figures
importlib.reload(utils.figures)

dataloader = DataLoader(dexed_dataset, config.train.minibatch_size, shuffle=True, num_workers=8, pin_memory=False)
dataloader_iter = iter(dataloader)

In [42]:
sample = next(dataloader_iter)
spectrogram, params, sample_info = sample


_ = utils.figures.plot_spectrograms(spectrogram, specs_recons=None, presets_UIDs=sample_info[:, 0],
                                    print_info=False, plot_error=False, add_colorbar=True, max_nb_specs=3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# Corresponding audio - insert UID
UID = sample_info[0, 0].item()  # Change first index to retrieve the sound corresponding to a spectrogram
print("Preset {}".format(UID))
x_wav, Fs = dataset.DexedDataset.get_wav_file(UID, dexed_dataset.midi_note, dexed_dataset.midi_velocity)
Audio(x_wav, rate=Fs)

In [ ]:
import torch
import os
# dataloader test
dexed_dataloader = torch.utils.data.DataLoader(dexed_dataset, batch_size=128, shuffle=False,
                                               num_workers=8)
sample = None
for i, sample in enumerate(dexed_dataloader):
    if i%50 == 0:
        print("batch {}/{}".format(i, len(dexed_dataloader)))
print(sample[0].size())

# Stats on all data spectrograms

Si on veut environ 100dB de dynamic range sur tout le dataset, il semble qu'il faut fixer le min absolu à environ -140dB.

Dexed : le max absolu d'un pixel de spectrogramme est environ -30dB sur le dataset complet, ce qui paraît faible mais est juste. La classe Spectrogram donne bien environ -6dB pour le max du spectre (fréq. positives) d'un sinus pur.

In [10]:
import pandas as pd
import json

with open(dexed_dataset._get_spectrogram_stats_file(), 'r') as f:
    print("Whole-dataset spectrogram stats: {} dB".format(json.load(f)))

spectrogram_stats_df = pd.read_csv(dexed_dataset._get_spectrogram_full_stats_file())
fig, axes = plt.subplots(4, 1, figsize=(9, 8), sharex=True)
n_bins = 50
for i, col in enumerate(['min', 'max', 'mean', 'std']):
    spectrogram_stats_df.hist(column=col, ax=axes[i], bins=n_bins)
    axes[i].set(title='spectrogram {} (dB)'.format(col))
fig.tight_layout()

Whole-dataset spectrogram stats: {'min': -120.0, 'max': -8.434515953063965, 'mean': -96.25504531409534, 'std': 20.20094461679582} dB


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …